### CALCULATE FEATURES BASED ON FOOD INSPECTION DATA

In [1]:
import numpy as np
import pandas as pd
import os.path

root_path = os.path.dirname(os.getcwd())

# Load food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

# Create basis for model_data
data = inspections.loc[:, ["inspection_id", "license", "inspection_date", "facility_type"]]

In [2]:
# Create pass / fail flags
data["pass_flag"] = inspections.results.apply(lambda x: 1 if x == "Pass" else 0)
data["fail_flag"] = inspections.results.apply(lambda x: 1 if x == "Fail" else 0)

In [3]:
# Create risk flags
data["risk_1"] = inspections.results.apply(lambda x: 1 if x == "Risk 1 (High)" else 0)
data["risk_2"] = inspections.results.apply(lambda x: 1 if x == "Risk 2 (Medium)" else 0)
data["risk_3"] = inspections.results.apply(lambda x: 1 if x == "Risk 3 (Low)" else 0)

In [4]:
# Load violation data
values = pd.read_csv(os.path.join(root_path, "DATA/violation_values.csv"))
counts = pd.read_csv(os.path.join(root_path, "DATA/violation_counts.csv"))

# Merge with violation data, filtering missing data
data = pd.merge(data, values, on="inspection_id")
data = pd.merge(data, counts, on="inspection_id")

In [6]:
# Sort inspections by date
license_groups = data.sort_values("inspection_date").groupby("license")

# Find previous inspections by shifting each sorted group
past_data = license_groups.shift(1)

In [7]:
# Add past fails, with 0 for first inspections
data["past_fail"] = past_data.fail_flag.fillna(0)

# Add past violation counts, with 0 for first records
data["past_critical"] = past_data.critical_count.fillna(0)
data["past_serious"] = past_data.serious_count.fillna(0)
data["past_minor"] = past_data.minor_count.fillna(0)

In [8]:
# Select past violation values, remove past inspection id
past_values = past_data[values.columns].drop("inspection_id", axis=1).add_prefix("p")

# Add past values to model data, with 0 for first records
data = data.join(past_values.fillna(0))

In [9]:
# Calculate time since previous inspection
deltas = pd.to_datetime(data.inspection_date) - pd.to_datetime(past_data.inspection_date)

# Add years since previous inspection, with 2 for first records
data["time_since_last"] = deltas.apply(lambda x: x.days / 365.25).fillna(2)

In [10]:
# Check if first record
data["first_record"] = past_data.inspection_id.map(lambda x: 1 if pd.isnull(x) else 0)

### CALCULATE FEATURES BASED ON BUSINESS LICENSE DATA

In [11]:
# Load business license data
licenses = pd.read_csv(os.path.join(root_path, "DATA/business_licenses.csv"))

/home/russell/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,23,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# Business licenses have numbers on end preventing simple match
# so using street number instead
def get_street_number(address):
    return address.split()[0]

licenses["street_number"] = licenses.address.apply(get_street_number)
inspections["street_number"] = inspections.address.apply(get_street_number)

In [11]:
# Match based on DBA name and street number
venue_matches = pd.merge(inspections, licenses, left_on=["dba_name", "street_number"], right_on=["doing_business_as_name", "street_number"])

# Match based on license numbers
license_matches = pd.merge(inspections, licenses, left_on="license", right_on="license_number")

# Join matches, reset index, drop duplicates
matches = venue_matches.append(license_matches, sort=False)
matches.reset_index(drop=True, inplace=True)
matches.drop_duplicates(["inspection_id", "id"], inplace=True)

# Restrict to matches where inspection falls within license period
matches = matches.loc[matches.inspection_date.between(matches.license_start_date, matches.expiration_date)]

In [12]:
# Select retail food establishment inspection IDs
retail = matches.loc[matches.license_description == "Retail Food Establishment", ["inspection_id"]]
retail.drop_duplicates(inplace=True)

# FILTER: ONLY CONSIDER INSPECTIONS MATCHED WITH RETAIL LICENSES
data = pd.merge(data, retail, on="inspection_id")

In [13]:
# Convert dates to datetime format
matches.inspection_date = pd.to_datetime(matches.inspection_date)
matches.license_start_date = pd.to_datetime(matches.license_start_date)

def get_age_data(group):
    min_date = group.license_start_date.min()
    deltas = group.inspection_date - min_date
    group["age_at_inspection"] = deltas.apply(lambda x: x.days / 365.25)
    return group[["inspection_id", "age_at_inspection"]]

# Calculate (3 mins), drop duplicates
age_data = matches.groupby("license").apply(get_age_data).drop_duplicates()

In [14]:
# Merge in age_at_inspection
data = pd.merge(data, age_data, on="inspection_id", how="left")

In [15]:
# Translate categories to snake-case titles
categories = {
    "Consumption on Premises - Incidental Activity": "consumption_on_premises_incidental_activity",
    "Tobacco": "tobacco",
    "Package Goods": "package_goods",
    "Limited Business License": "limited_business_license",
    "Outdoor Patio": "outdoor_patio",
    "Public Place of Amusement": "public_place_of_amusement",
    "Children's Services Facility License": "childrens_services_facility_license",
    "Tavern": "tavern",
    "Regulated Business License": "regulated_business_license",
    "Filling Station": "filling_station",
    "Caterer's Liquor License": "caterers_liquor_license",
    "Mobile Food License": "mobile_food_license"
}

# Create binary markers for license categories
def get_category_data(group):
    df = group[["inspection_id"]].iloc[[0]]
    for category in group.license_description:
        if category in categories:
            df[categories[category]] = 1
    return df
    
# group by inspection, get categories (2 mins)
category_data = matches.groupby("inspection_id").apply(get_category_data)

# Reset index, set absent categories to 0
category_data.reset_index(drop=True, inplace=True)
category_data.fillna(0, inplace=True)

In [16]:
# Merge in category data, fill nan with 0
data = pd.merge(data, category_data, on="inspection_id", how="left").fillna(0)

### ATTACH KDE DATA

In [17]:
# Load violation data
burglary_kde = pd.read_csv(os.path.join(root_path, "DATA/burglary_kde.csv"))
cart_kde = pd.read_csv(os.path.join(root_path, "DATA/cart_kde.csv"))
complaint_kde = pd.read_csv(os.path.join(root_path, "DATA/complaint_kde.csv"))

# FILTER: only consider data since 2012 (with good kde data)
data = pd.merge(data, burglary_kde, on="inspection_id")
data = pd.merge(data, cart_kde, on="inspection_id")
data = pd.merge(data, complaint_kde, on="inspection_id")

### ATTACH WEATHER DATA

In [18]:
# Load weather data
weather = pd.read_csv(os.path.join(root_path, "DATA/weather.csv"))

# Merge weather data with model data
data = pd.merge(data, weather, on="inspection_id")

In [20]:
# Save Result
data.to_csv(os.path.join(root_path, "DATA/model_data.csv"), index=False)